In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import multibind as mb
import numpy as np
import pandas as pd
import torch
import bindome as bd
bd.constants.ANNOTATIONS_DIRECTORY = '/home/johanna/ICB/annotations'
# mb.models.MultiBind
import torch.optim as topti
import torch.utils.data as tdata
import matplotlib.pyplot as plt
import logomaker

# Use a GPU if available, as it should be faster.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

Using device: cpu


In [25]:
n_rounds = 1
data = pd.read_csv('../data/countTable.0.CTCF_r3.tsv.gz', sep='\t', header=None)
data.columns = ['seq'] + [i for i in range(n_rounds+1)]
data.index = data['seq']
del data['seq']
data = data.sample(n=1000)
# data.index = range(len(data))
dataset = mb.datasets.SelexDataset(data, n_rounds=n_rounds)
train = tdata.DataLoader(dataset=dataset, batch_size=128, shuffle=True)
# train_test = tdata.DataLoader(dataset=dataset, batch_size=1, shuffle=False)

In [26]:
data

,0,1
seq,,
ATATTAGTTATGTCGCGTCTGCTATAGATG,1,0
AATGTGCTGGTGCTTTTGTTATAGTGAATA,1,0
AGAAGTGGTCGTGGGTTGAGCTGTTGGAGG,1,0
CTTGTAGGTTATAACTTTACGCTGAATTTG,1,0
GGTAGCTGAGCTCAGAGCTAGTAACTAATG,1,0
...,...,...
GATGCCCGGGCACAAGGTGTACGGTGCGGG,0,1
ACGCGGAGCAAATGCTGGTTCATTAAAGGT,1,0
AGAAATAGGCTAATGCAGAGTATACGCAGG,1,0


In [27]:
model = mb.models.Multibind(kernels=[0, 14, 12], datatype="selex", n_rounds=1, n_batches=1).to(device)
l2 = []
optimiser = topti.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
# optimiser = topti.LBFGS(model.parameters())
criterion = mb.tl.PoissonLoss()

In [28]:
mb.tl.train_network(model, train, device, optimiser, criterion, reconstruction_crit=None, num_epochs=500, early_stopping=10, log_each=10, verbose=2)

optimizing using <class 'torch.optim.adam.Adam'> and <class 'multibind.tl.loss.PoissonLoss'> n_epochs 500 early_stopping 10
lr= 0.001, weight_decay= 0.0001, dir weight= 0
Epoch: 11, Loss: 1.663309 , best epoch: 9 secs per epoch: 0.062 s
Epoch: 21, Loss: 1.332551 , best epoch: 19 secs per epoch: 0.060 s
Epoch: 31, Loss: 1.077441 , best epoch: 29 secs per epoch: 0.060 s
Epoch: 41, Loss: 0.933565 , best epoch: 39 secs per epoch: 0.059 s
Epoch: 51, Loss: 0.874410 , best epoch: 49 secs per epoch: 0.060 s
Epoch: 61, Loss: 0.854477 , best epoch: 59 secs per epoch: 0.059 s
Epoch: 71, Loss: 0.848061 , best epoch: 69 secs per epoch: 0.058 s
Epoch: 81, Loss: 0.846159 , best epoch: 79 secs per epoch: 0.057 s
Epoch: 91, Loss: 0.845833 , best epoch: 88 secs per epoch: 0.056 s
Epoch: 101, Loss: 0.845492 , best epoch: 99 secs per epoch: 0.055 s
Epoch: 111, Loss: 0.845479 , best epoch: 101 secs per epoch: 0.055 s
Epoch: 112, Loss: 0.8456 , best epoch: 101 secs per epoch: 0.055 s
early stop!
total time: